In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from deep4deep import text_processing

In [ ]:
import pandas as pd

# (re)construct an input

In [24]:
#ScanR
# given an id, get the 'description' in english, or in French if no english

from googletrans import Translator #https://pypi.org/project/googletrans/
# or 
# from translate import Translator

def scanr_from_id_to_en_description(company_id=833714694):
    base_url = "https://scanr-api.enseignementsup-recherche.gouv.fr/api/v2/structures/structure/"
    url = base_url + str(company_id)
    response = requests.get(url).json()

    # if no english, send a request to Google translation API
    # originally using googletrans, but problem these days: https://stackoverflow.com/questions/52455774/googletrans-stopped-working-with-error-nonetype-object-has-no-attribute-group
    # therefore using translate: https://pypi.org/project/translate/

    english = response['description']['en']
    if not english:
        print(f"for company {company_id}, could only retrieve French description. Sending to translation.")
        french = response['description']['fr']
        #======= for googletrans module 
        # currently having a documented issue sending error: 
        # AttributeError: 'NoneType' object has no attribute 'group'
        #translator = Translator() 
        #english = translator.translate(french, dest='en', src='fr') 

        #======= for translate module
        #translator = Translator(to_lang='en', from_lang='fr',) #with a limited number of calls a day
        #english = translator.translate(french) 

        # if no API available: return the french text until a solution is found
        print(f"for company {company_id}, could not get translation. Keeping French.")
    englishenglish = french 
    
    return english


scanr_from_id_to_en_description(483185807)

AttributeError: 'NoneType' object has no attribute 'group'

In [21]:
#ScanR
# given a name, return an id (POST)
# written mail to API team
# written stack overflow post: https://stackoverflow.com/questions/65106756/using-the-scanr-api-from-gouv-fr-for-a-post-request-that-given-a-company-name

In [128]:
url_structures = "https://scanr-api.enseignementsup-recherche.gouv.fr/api/v2/structures/search"
my_query = "carbon\ waters"

params = {
   "pageSize": 12,
   "query": my_query
}

scanr_output = requests.post(url_structures, json=params).json()
scanr_output

{'request': {'query': 'carbon\\ waters',
  'page': 0,
  'pageSize': 12,
  'lang': 'default',
  'searchFields': [],
  'filters': {},
  'sourceFields': [],
  'aggregations': {}},
 'total': 2887,
 'results': [{'highlights': [{'type': 'label.fr',
     'value': '<strong>CARBON</strong> <strong>WATERS</strong>'},
    {'type': 'label.default',
     'value': '<strong>CARBON</strong> <strong>WATERS</strong>'},
    {'type': 'description.fr',
     'value': '<strong>Carbon</strong> <strong>Waters</strong> est une société de graphène. Nous avons développé et breveté un procédé unique pour produire'},
    {'type': 'description.default',
     'value': '<strong>Carbon</strong> <strong>Waters</strong> est une société de graphène. Nous avons développé et breveté un procédé unique pour produire'},
    {'type': 'alias',
     'value': '<strong>CARBON</strong> <strong>WATERS</strong>'},
    {'type': 'websites.webPages.content',
     'value': 'Unit\nFormulations Unit\nFind solutions\nContact\n<strong>Carbon<

In [113]:
id = scanr_output['results'][0]['value']['id']
id

'833714694'

In [114]:
scanr_from_id_to_en_description(id)

"Carbon Waters est une société de graphène. Nous avons développé et breveté un procédé unique pour produire du graphène monocouche dans l'eau. Contrairement aux dispersions d'oxyde de graphène, notre graphène est pur, stable et sûr à utiliser dans des contextes de recherche et industriels."

In [116]:
scanr_output.keys()

dict_keys(['request', 'total', 'results', 'facets'])

In [118]:
scanr_output['request'].keys()

dict_keys(['query', 'page', 'pageSize', 'lang', 'searchFields', 'filters', 'sourceFields', 'aggregations'])

In [120]:
type(scanr_output['total'])

int

In [157]:
type(scanr_output['results'])
len(scanr_output['results'])
# results comes as a list with number of items equal to 'pageSize'

print("What's in list 'results'? A number of dict items with the same 2 keys\n")
      
for i, item in enumerate(scanr_output['results'][:3]):
    print(f"item {i} in results has keys: {item.keys()}")
    # results yields pageSize items:
        # of kind dict
        # with 2 keys: highlights (list) and value (dict)
        # each being a list
    print(f"length of list highlights: {len(item['highlights'])}")
    print(f"random elements of highlights: {item['highlights'][len(item['highlights'])-1]}\n")

    print(f"what's in dict 'value': {item['value'].keys()}")
    print(f"in 'id' inside values: {item['value']['id']}")
    print("\n")
    


What's in list 'results'? A number of dict items with the same 2 keys

item 0 in results has keys: dict_keys(['highlights', 'value'])
length of list highlights: 6
random elements of highlights: {'type': 'websites.webPages.content', 'value': 'Unit\nFormulations Unit\nFind solutions\nContact\n<strong>Carbon</strong> <strong>Waters</strong>\nPioneering an innovative graphene form\nOur Team'}

what's in dict 'value': dict_keys(['id', 'kind', 'label', 'nature', 'isFrench', 'address', 'level'])
in 'id' inside values: 833714694


item 1 in results has keys: dict_keys(['highlights', 'value'])
length of list highlights: 4
random elements of highlights: {'type': 'publications.publication.title.default', 'value': 'Early spring mesopelagic <strong>carbon</strong> remineralization and transfer efficiency in the'}

what's in dict 'value': dict_keys(['id', 'kind', 'label', 'acronym', 'nature', 'isFrench', 'address', 'level'])
in 'id' inside values: 200812834V


item 2 in results has keys: dict_keys([

In [43]:
from deep4deep import text_retrieval

In [40]:
from bs4 import BeautifulSoup

def get_meta_description(website):
    '''
    given a website url, returns the description from the meta tags
    returns: string
    '''
    website = 'https://www.carbon-waters.com/'

    response = requests.get(website)
    soup = BeautifulSoup(response.content, "html.parser")
    description = soup.find("meta", property="og:description")["content"]

    return description
get_meta_description("www.carbon-waters.com")

'Carbon Waters is a graphene company, we have developed and patented a unique process to produce very high-quality graphene in water.'

In [45]:
text_retrieval.get_meta_description("http://www.carbon-waters.com")

'Carbon Waters is a graphene company, we have developed and patented a unique process to produce very high-quality graphene in water.'

In [1]:
import deep4deep
import os

im in init.py


In [14]:
import requests
import re

def import_dealroom_news(company_id=214127, n_news=10):
    '''
    queries the Dealroom API for news and returns a clean string, in English, 
    comprising up to n_news most recent news. For each, are returned:
    title, summary, name of feed, content with all html tags removed, names of categories
    company_id: integer, Dealroom company id
    n_news: max quantity of news to retrieve
    returns: as one single concatenated string, empty if the API returned an error
    '''
    URL = 'https://api.dealroom.co/api/v1'

    APIKEY = os.getenv('DEALROOMAPIKEY')
    if not APIKEY:
        print("The API key could not be retrieved correctly")
        
    response = requests.get(
                    url = f'{URL}/companies/{company_id}/news',\
                    auth = (APIKEY, '')
                    )

    return_string = []
    try: 
        json = response.json()
        for item in json['items']:
            return_string.append(item['title'])    
            return_string.append(item['summary'])
            return_string.append(item['feed']['name'])
            content = re.sub(r"<.+?>", '', item['content']) # remove all HTML tags from content
            return_string.append(content)
            categories_names = ' '.join([i['name'] for i in item['categories']])
            return_string.append(categories_names) 
        return return_string
    except:
        print("The Dealroom news API could not be queried")
        return return_string

In [15]:
import_dealroom_news(214127)

["Genfit's NASH drug fails a closely-watched PhIII showdown, adding one more setback to a plagued field",
 "Genfit's bid to make R&D history came to a crushing close on Monday as the French biotech acknowledged that its Phase III study of elafibranor failed to distinguish itself from placebo in treating NASH. The biotech had tried to prove that the drug would resolve NASH without fibrosis in a",
 'Endpoints News',
 'Genfit’s bid to make R&amp;D history came to a crushing close on Monday as the French biotech acknowledged that its Phase III study of elafibranor failed to distinguish itself from placebo in treating NASH.\nPascal Prigent\nThe biotech had tried to prove that the drug would resolve NASH without fibrosis in a population of more than 1,000 patients. But the data came up with a 19.2% response rate for the drug arm compared to a 14.7% rate for the placebo.\nGenfit CEO Pascal Prigent put down the “disappointing” result to a high placebo response, though the drug arm didn’t appea

Steps: 

+ Create a df with id, name, site, metadata, dealroom news
+ 